In [2]:
import numpy as np
import pandas as pd
import xlsxwriter 
import openpyxl 
from openpyxl.styles import Border, Font, Alignment, Side

green_font = Font(name='Century',
                 size=12,
                 color='008000')

red_font = Font(name='Century',
                 size=12,
                 color='FF0000')

blue_font = Font(name='Century',
                 size=12,
                 color='0000FF')

bold_font = Font(name='Century',
                 bold=True,
                 size=12)

thin = Side(border_style="thin")
border = Border(top=thin, left=thin, right=thin, bottom=thin)

align = Alignment(horizontal="center")

master_roll = pd.read_csv("sample_input/master_roll.csv")
 
responses = pd.read_csv("sample_input/responses.csv")

roll_nos = list(master_roll['roll'].iloc[:])


responses['Roll Number'] = responses['Roll Number'].str.upper()

In [9]:
p = 5
n = -1

In [29]:
def excel_creator(output_path,correct_ans):
	workbook = xlsxwriter.Workbook(output_path)
	worksheet = workbook.add_worksheet('quiz')

	worksheet.set_column(0,5,17)
	worksheet.set_row(4,22)
	worksheet.insert_image('A1', 'logo.png',{'x_scale':620/853, 'y_scale':82/126})

	merge_format = workbook.add_format({
                                        'bold': 1, 
                                        'align': 'center',
                                        'underline': 1,
                                        'font_size':16,
                                        'font_name':'Century'})
	right_format = workbook.add_format({
                                        'align':'right',
                                        'font_name':'Century',
                                        'font_size':12 })
	bold_format = workbook.add_format({
                                        'bold':1,
                                        'font_name':'Century',
                                        'font_size':12 })
	header_format = workbook.add_format({
                                        'bold':1,
                                        'align':'center',
                                        'font_size':12,
                                        'font_name':'Century',
                                        'border':1})
	border_format = workbook.add_format({
                                        'border':1
                                        })
	blue_format = workbook.add_format({
                                        'border':1,
                                        'font_name':'Century',
                                        'font_size':12,
                                        'font_color':'#0000FF',
                                        'align':'center'})
	black_format = workbook.add_format({
                                        'border':1,
                                        'font_name':'Century',
                                        'font_size':12, 
                                        'align':'center'})

	worksheet.merge_range('A5:E5', 'Marks Sheet', merge_format)
	worksheet.write('A6','Name:',right_format)
	worksheet.write('A7','Roll Number:',right_format)
	worksheet.write('D6','Exam:',right_format)
	worksheet.write('E6','quiz',bold_format)
	worksheet.write('A9',' ',header_format)
	worksheet.write('B9','Right',header_format)
	worksheet.write('C9','Wrong',header_format)
	worksheet.write('D9','Not Attempt',header_format)
	worksheet.write('D12',' ',header_format)
	worksheet.write('E9','Max',header_format)
	worksheet.write('E10',28,black_format)
	worksheet.write('E11',' ',header_format)
	worksheet.write('A10','No.',header_format)
	worksheet.write('A11','Marking',header_format)
	worksheet.write('A12','Total',header_format)
	worksheet.write('A15','Student Ans',header_format)
	worksheet.write('B15','Correct Ans',header_format)
	worksheet.write('D15','Student Ans',header_format)
	worksheet.write('E15','Correct Ans',header_format)

	for i in range(25):
		worksheet.write('B'+str(16+i),correct_ans[i],blue_format)

	for i in range(len(correct_ans)-25):
		worksheet.write('E'+str(16+i),correct_ans[25+i],blue_format)

	workbook.close()
     

def marksheet_creator(roll_number, responses, master_roll):

	output_path = "marksheets/" + roll_number + ".xlsx"

	correct_ans = list(responses[responses['Roll Number']=='ANSWER'].loc[:,'Unnamed: 7':'Unnamed: 34'].iloc[0])

	given_response = list(responses[responses['Roll Number']==roll_number].loc[:,'Unnamed: 7':'Unnamed: 34'].iloc[0])

	excel_creator(output_path, correct_ans)
	workbook = openpyxl.load_workbook(output_path)
	worksheet = workbook.get_sheet_by_name('quiz')
	worksheet['B6'] =  str(master_roll[master_roll['roll']==roll_number]['name'].iloc[0])
	worksheet['B6'].font = bold_font
	worksheet['B7'] = roll_number
	worksheet['B7'].font = bold_font
	correct = 0
	wrong = 0
	not_attempted = 0
	for j in range(len(given_response)):
		if(pd.isnull(given_response[j])):
			not_attempted += 1
		elif(given_response[j]==correct_ans[j]):
			correct += 1
			if(j<25):
				worksheet['A'+str(16+j)] = correct_ans[j] 
				worksheet['A'+str(16+j)].font = green_font
				worksheet['A'+str(16+j)].alignment = align
				worksheet['A'+str(16+j)].border = border
			else:
				worksheet['D'+str(16+j-25)] = correct_ans[j]
				worksheet['D'+str(16+j-25)].font = green_font
				worksheet['D'+str(16+j-25)].alignment = align
				worksheet['D'+str(16+j-25)].border = border
		else:
			wrong += 1
			if(j<25):
				worksheet['A'+str(16+j)] = given_response[j]
				worksheet['A'+str(16+j)].font = red_font
				worksheet['A'+str(16+j)].alignment = align
				worksheet['A'+str(16+j)].border = border
			else:
				worksheet['D'+str(16+j-25)] = given_response[j]
				worksheet['D'+str(16+j-25)].font = red_font
				worksheet['D'+str(16+j-25)].alignment = align
				worksheet['D'+str(16+j-25)].border = border

	# st.write('correct: {}, wrong: {}, not_attempted: {} '.format(correct,wrong,not_attempted))
	worksheet['B10'] = correct
	worksheet['B10'].font = green_font
	worksheet['B10'].alignment = align
	worksheet['B10'].border = border
	worksheet['B11'] =  p
	worksheet['B11'].font = green_font
	worksheet['B11'].alignment = align
	worksheet['B11'].border = border
	worksheet['B12'] =  correct*p
	worksheet['B12'].font = green_font
	worksheet['B12'].alignment = align
	worksheet['B12'].border = border

	worksheet['C10'] = wrong
	worksheet['C10'].font = red_font
	worksheet['C10'].alignment = align
	worksheet['C10'].border = border
	worksheet['C11'] = n
	worksheet['C11'].font = red_font
	worksheet['C11'].alignment = align
	worksheet['C11'].border = border
	worksheet['C12'] = wrong*n
	worksheet['C12'].font = red_font
	worksheet['C12'].alignment = align
	worksheet['C12'].border = border

	worksheet['D10'] = not_attempted
	worksheet['D10'].font = Font(name='Century',size=12)
	worksheet['D10'].alignment = align 
	worksheet['D10'].border = border
	worksheet['D11'] = 0
	worksheet['D11'].font = Font(name='Century',size=12)
	worksheet['D11'].alignment = align
	worksheet['D11'].border = border

	worksheet['E12'] = str(correct*p+wrong*n)+'/'+str(len(correct_ans)*p)
	worksheet['E12'].font = blue_font
	worksheet['E12'].alignment = align
	worksheet['E12'].border = border

	workbook.save(output_path)

	return [correct,wrong,not_attempted]


 
def generate_rollwise(): 
    Summary = []
    score = []
    
    if('ANSWER' in list(responses['Roll Number'])):
        
        for i in range(len(roll_nos)):
            if(roll_nos[i] in list(responses['Roll Number'])):
                correct,wrong,not_attempted = marksheet_creator(roll_nos[i],responses, master_roll)
                Summary.append([correct,wrong,not_attempted])
                score.append(str(correct*p+wrong*n)+'/'+str(140))
                print(roll_nos[i],[correct,wrong,not_attempted])
            else:
                output_path = "marksheets/" + roll_nos[i] + ".xlsx"
                workbook = xlsxwriter.Workbook(output_path)
                worksheet = workbook.add_worksheet('quiz')
                workbook.close()
                print('Absent')
#     print(Summary)
    return Summary,score
     
 

In [30]:
Summary, score = generate_rollwise()

C:\Users\aravind reddy\AppData\Local\Temp/ipykernel_16712/3182192184.py:84: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  worksheet = workbook.get_sheet_by_name('quiz')


ANSWER [28, 0, 0]
1401ME61 [11, 16, 1]
1401CS40 [20, 8, 0]
1401EE17 [5, 12, 11]
1401ME10 [8, 13, 7]
1401ME69 [9, 12, 7]
1401CS31 [20, 7, 1]
1401CS15 [18, 2, 8]
1401EE32 [7, 11, 10]
1401CE04 [21, 1, 6]
1401CS68 [18, 7, 3]
1401CE61 [26, 2, 0]
1401ME80 [4, 0, 24]
1401ME83 [9, 8, 11]
1401ME52 [12, 16, 0]
1401EE58 [20, 3, 5]
1401EE56 [5, 6, 17]
1401ME07 [21, 1, 6]
1401CS63 [19, 2, 7]
1401ME42 [22, 5, 1]
1401ME71 [16, 5, 7]
1401CB51 [27, 1, 0]
1401EE15 [13, 4, 11]
1401CS66 [27, 1, 0]
1401CS44 [28, 0, 0]
1401CS14 [25, 2, 1]
1401EE27 [16, 1, 11]
1401EE36 [14, 5, 9]
1401CS78 [28, 0, 0]
1401CE47 [14, 1, 13]
1401CE29 [15, 8, 5]
1401ME11 [21, 3, 4]
1401ME45 [22, 2, 4]
1401CB02 [17, 3, 8]
1401CB16 [25, 1, 2]
1401MM25 [23, 1, 4]
1401EE67 [21, 3, 4]
1401CE25 [21, 1, 6]
1401CB37 [9, 6, 13]
1401CS67 [19, 6, 3]
1401CS74 [15, 3, 10]
1401ME76 [19, 2, 7]
1401EE20 [10, 0, 18]
1401EE53 [22, 1, 5]
1401EE18 [14, 7, 7]
1401CB35 [13, 1, 14]
1401MM04 [8, 10, 10]
1401ME06 [20, 4, 4]
1401EE68 [11, 17, 0]
1401CB19 [

In [31]:
score

['140/140',
 '39/140',
 '92/140',
 '13/140',
 '27/140',
 '33/140',
 '93/140',
 '88/140',
 '24/140',
 '104/140',
 '83/140',
 '128/140',
 '20/140',
 '37/140',
 '44/140',
 '97/140',
 '19/140',
 '104/140',
 '93/140',
 '105/140',
 '75/140',
 '134/140',
 '61/140',
 '134/140',
 '140/140',
 '123/140',
 '79/140',
 '65/140',
 '140/140',
 '69/140',
 '67/140',
 '102/140',
 '108/140',
 '82/140',
 '124/140',
 '114/140',
 '102/140',
 '104/140',
 '39/140',
 '89/140',
 '72/140',
 '93/140',
 '50/140',
 '109/140',
 '63/140',
 '64/140',
 '30/140',
 '96/140',
 '38/140',
 '70/140',
 '129/140',
 '124/140',
 '128/140',
 '44/140',
 '109/140',
 '72/140',
 '33/140',
 '103/140',
 '103/140',
 '114/140',
 '72/140',
 '119/140',
 '50/140',
 '122/140',
 '67/140',
 '94/140',
 '62/140',
 '129/140',
 '140/140',
 '125/140',
 '50/140',
 '103/140',
 '38/140',
 '116/140',
 '116/140',
 '123/140',
 '76/140',
 '100/140',
 '93/140',
 '112/140',
 '128/140',
 '110/140',
 '129/140',
 '105/140',
 '128/140',
 '112/140',
 '112/140',
 

In [40]:
responses['statusAns'] = Summary
responses['Score_After_Negative'] = score
responses.columns  = ['Timestamp', 'Email address', 'Google_Score', 'Name', 'IITP webmail',
       'Phone (10 digit only)', 'Roll Number', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'statusAns', 'Score_After_Negative'] 

In [42]:
responses

,Timestamp,Email address,Google_Score,Name,IITP webmail,Phone (10 digit only),Roll Number,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,statusAns,Score_After_Negative
0,07-07-2021 14:10,dsdeeptishakya777@gmail.com,140 / 140,Deepti Shakya,deepti_1921cs14@iitp.ac.in,8377094336,ANSWER,Option A,Option D,Option B,...,Option A,Option A,Option A,Option D,Option D,Option A,Option C,Option D,"[28, 0, 0]",140/140
1,07-08-2021 12:18,ellen_booth4939@atink.com,55 / 140,Ellen Booth,ellen_booth9706@iitp.ac.in,7785862435,1401ME61,Option A,Option D,Option B,...,Option D,Option B,Option B,Option C,Option B,Option A,Option A,Option B,"[11, 16, 1]",39/140
2,07-08-2021 12:18,rachael_selby9175@acrit.org,100 / 140,Rachael Selby,rachael_selby7861@iitp.ac.in,9984503025,1401CS40,Option A,Option D,Option B,...,Option A,Option B,Option A,Option D,Option B,Option A,Option C,Option D,"[20, 8, 0]",92/140
3,07-08-2021 12:19,rowan_knott5318@deons.tech,25 / 140,Rowan Knott,rowan_knott132@iitp.ac.in,9799319607,1401EE17,Option A,NaN,Option C,...,Option B,Option A,Option B,Option B,NaN,Option A,NaN,Option A,"[5, 12, 11]",13/140
4,07-08-2021 12:19,abdul_rowe197@muall.tech,40 / 140,Abdul Rowe,abdul_rowe997@iitp.ac.in,9131454766,1401ME10,Option D,Option D,Option B,...,Option B,NaN,Option B,NaN,Option C,Option C,Option B,Option D,"[8, 13, 7]",27/140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,07-08-2021 12:23,valentina_snell4052@supunk.biz,45 / 140,Valentina Snell,valentina_snell5918@iitp.ac.in,8004319883,1401CE19,NaN,Option D,NaN,...,Option A,NaN,NaN,NaN,NaN,Option A,Option C,NaN,"[9, 2, 17]",43/140
390,07-08-2021 12:23,denny_whitmore6196@infotech44.tech,85 / 140,Denny Whitmore,denny_whitmore566@iitp.ac.in,7480889813,1401CE01,Option A,Option D,NaN,...,NaN,Option B,Option A,Option D,NaN,Option A,Option C,Option D,"[17, 6, 5]",79/140
391,07-08-2021 12:24,caitlyn_yarwood4974@deavo.com,105 / 140,Caitlyn Yarwood,caitlyn_yarwood7969@iitp.ac.in,9145935787,1401EE50,Option A,Option D,Option B,...,NaN,Option B,Option A,Option D,Option B,Option A,Option C,Option D,"[21, 3, 4]",102/140
392,07-08-2021 12:26,nathan_ebbs6353@bretoux.com,55 / 140,Nathan Ebbs,nathan_ebbs4812@iitp.ac.in,8795972438,1401CB64,NaN,NaN,NaN,...,NaN,NaN,Option A,NaN,Option D,Option A,Option C,NaN,"[11, 0, 17]",55/140


In [45]:
responses.to_excel("concise_marksheet.xlsx", sheet_name='concise_marksheet', index=False)